In [1]:
from pprint import pp
import datetime
import argparse
from pathlib import Path
import sqlite3
import traceback
import json
import re
from track_records.data_man import *
from prettytable import PrettyTable

def adapt_date_iso(val):
    """Adapt datetime.date to ISO 8601 date."""
    return val.isoformat()
sqlite3.register_adapter(datetime.date, adapt_date_iso)
def convert_date(val):
    """Convert ISO 8601 date to datetime.date object."""
    return datetime.date.fromisoformat(val)
sqlite3.register_converter('date', convert_date)


In [9]:
results_2024 = execute_named_query('data/track_records.sqlite',
    "data/queries.sql",
    "get_all_athletes_from_team_in_year<>", ("Saint Joseph Catholic School", "2024"))
results_2025 = execute_named_query('data/track_records.sqlite',
    "data/queries.sql",
    "get_all_athletes_from_team_in_year<>", ("Saint Joseph Catholic School", "2025"))

names_2025 = [athlete['athlete_name'] for athlete in results_2025]
names_2024 = [athlete['athlete_name'] for athlete in results_2024]
# pp(names_2024)
# pp(names_2025)

# merge the names list
names_2024_2025 = list(set(names_2024 + names_2025))
pp(names_2024_2025)

['Jude Cummins',
 'Howard Valdiviez',
 'Julia Alder',
 'Daniel Antinora',
 'Faith Williams',
 'Paige Carlson',
 'Joey Peacock',
 'Saint Joseph Catholic School m4x100m relay team',
 'Elizabeth Davis',
 'Elijah Ruybal',
 'Emerson Harres',
 'Isaac Watkins',
 'Anja Scherpa',
 'Cora Bauer',
 'Patrick Lamb',
 'Emily Vignale',
 'Abbey Rojeski',
 'Saint Joseph Catholic School f4x100m relay team',
 'Morgan Carlson',
 'Mason Bauer',
 'Sienna Harden',
 'Sylvie Dellenbach',
 'Evelyn Mast',
 'Lucy Mast',
 'Vivian Bozek',
 'Marilyn Wade',
 'Saint Joseph Catholic School f4x200m relay team',
 'Colton Harres',
 'Felicity Williams',
 'Evelyn Davis']


Set the names I want to use.

In [3]:
# names = [
#  'Morgan Carlson',
#  'Cora Bauer',
#  'Paige Carlson',
#  'Felicity Williams',
#  'Abbey Rojeski',
#  'Emerson Harres',
#  'Marilyn Wade',
#  'Vivian Bozek',
#  'Lucy Mast',
#  'Daniel Antinora',
#  'Elijah Ruybal',
#  'Sienna Harden',
#  'Joey Peacock',
#  'Evelyn Mast']

In [4]:
athlete_records = execute_named_query('data/track_records.sqlite',
    "data/queries.sql",
    "get_records_from_all_athletes_on_team<>_in_year<>", ("Saint Joseph Catholic School", "2024"))
    # pp(athlete_records)

print(athlete_records)

[{'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls 100 meter dash', 'meet_date': '2024-04-26', 'location': 'Skyline High School', 'result': '15.30', 'result_sort': 15.3}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls 100 meter high hurdles', 'meet_date': '2024-04-26', 'location': 'Skyline High School', 'result': '22.99', 'result_sort': 22.99}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls 200 meter dash', 'meet_date': '2024-04-22', 'location': 'Skyline High School', 'result': '33.27', 'result_sort': 33.27}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls discus', 'meet_date': '2024-05-10', 'location': 'Skyline High School', 'result': '62-08.00', 'result_sort': -62.666666666666664}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls long jump', 'meet_date': '2024-05-10', 'location': 'Skyline High School', 'result': '11-10.00', 'result_sort': -11.833333333333334}, {'athlete_name': 'Abbey Rojeski', 'event_name': 'Girls shot put', 'meet_date': '2024-04-22'

Get the athletes for the names above.

In [10]:
placeholders = f"({', '.join(['?'] * len(names_2024_2025))})"
# print('placeholders = ', placeholders)
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_personal_records_for_athletes<>", tuple(names_2024_2025), PLACEHOLDER1=placeholders
)
# Create a PrettyTable object with the specified headings
table = PrettyTable(['Athlete Name', 'Event Name', 'Meet Date', 'Result', 'Location'])

# Add rows to the table
for record in athlete_records:
    table.add_row([record['athlete_name'], record['event_name'], record['meet_date'], record['result'], record['location']])

# Print the table
print(table)


+-------------------------------------------------+------------------------------+------------+----------+-----------------------------+
|                   Athlete Name                  |          Event Name          | Meet Date  |  Result  |           Location          |
+-------------------------------------------------+------------------------------+------------+----------+-----------------------------+
|                  Abbey Rojeski                  |     Girls 100 meter dash     | 2024-04-26 |  15.30   |     Skyline High School     |
|                  Abbey Rojeski                  |     Girls 200 meter dash     | 2024-04-22 |  33.27   |     Skyline High School     |
|                  Abbey Rojeski                  | Girls 100 meter high hurdles | 2024-04-26 |  22.99   |     Skyline High School     |
|                  Abbey Rojeski                  |       Girls long jump        | 2024-05-10 | 11-10.00 |     Skyline High School     |
|                  Abbey Rojeski         

Print out the report.

In [ ]:
report = PDFReport("track_records_report.pdf")
report.create_pdf(athlete_records)

In [ ]:
placeholders = f"({', '.join(['?'] * len(names_2024_2025))})"
# print('placeholders = ', placeholders)
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_results_from_athletes<>", tuple(names_2024_2025) + ("Saint Joseph Catholic School",), PLACEHOLDER1=placeholders
)

# Create a dictionary to store the data in the desired format
formatted_data = {}

# Process each record to organize data by athlete and event
for record in athlete_records:
    athlete = record['athlete_name']
    event = record['event_name']
    date = record['meet_date']
    result = record['result']

    # Initialize nested dictionaries if not already present
    if athlete not in formatted_data:
        formatted_data[athlete] = {}
    if event not in formatted_data[athlete]:
        formatted_data[athlete][event] = {}

    # Assign the result to the corresponding date column
    formatted_data[athlete][event][date] = result

# Create a PrettyTable object with dynamic date columns
dates = sorted({record['meet_date'] for record in athlete_records})
columns = ['Athlete Name', 'Event Name'] + dates
table = PrettyTable(columns)

# Populate the table with the formatted data
for athlete, events in formatted_data.items():
    for event, results in events.items():
        row = [athlete, event] + [results.get(date, '') for date in dates]
        table.add_row(row)

# Print the table
print(table)

+-------------------------------------------------+------------------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|                   Athlete Name                  |          Event Name          | 2019-04-06 | 2022-04-29 | 2022-05-03 | 2022-05-06 | 2022-05-10 | 2022-05-13 | 2023-04-12 | 2023-04-19 | 2023-04-26 | 2023-05-04 | 2024-04-22 | 2024-04-26 | 2024-05-04 | 2024-05-10 | 2025-04-05 |
+-------------------------------------------------+------------------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|                  Abbey Rojeski                  | Girls 100 meter high hurdles |            |            |            |            |            |            |      

In [11]:
placeholders = f"({', '.join(['?'] * len(names_2024_2025))})"
# print('placeholders = ', placeholders)
athlete_records = execute_named_query(
    'data/track_records.sqlite',
    "data/queries.sql",
    "get_results_from_athletes<>", tuple(names_2024_2025) + ("Saint Joseph Catholic School",), PLACEHOLDER1=placeholders
)

# Generate a PDF report for the results
report = PDFReport("athlete_results.pdf")
report.create_results_pdf(athlete_records)